# Piezometer analysis

## import libraries

In [ ]:
# -*- coding: utf-8 -*-
import pandas as pd
import matplotlib
import os
from IPython.display import HTML
import numpy as np
from datetime import datetime
from matplotlib import pyplot as plt

## Read datasets

In [ ]:
# piezo list
piezo = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'RAmont', 'RAval']

df_piezo_dict = {}

# read the excel file and add to the dictionary
for p in piezo:
    df_piezo_dict[p] = pd.read_excel('piezometry.xlsx', sheet_name=p)

# read piezo coordinates csv
csv_dir = os.path.join('data', 'bip', 'Piezometres')
piezo_coords = pd.read_csv(os.path.join(csv_dir, 'piezo_coordinates.csv'))

# read profile coordinates csv
profile_coords = pd.read_csv(os.path.join(csv_dir, 'profile_coordinates.csv'))

# read distance chenal piezo csv
distance_chenal_piezo = pd.read_csv(os.path.join(csv_dir, 'distance_chenal_piezo.csv'), index_col = 'fid')

# read berge piezo csv
berges_piezo = pd.read_csv(os.path.join(csv_dir, 'berges_piezo.csv'), index_col = 'fid')
# convert date to datetime
berges_piezo['date'] = pd.to_datetime(berges_piezo['date'])


## Add distance sum from RAmont to each different piezometer section and the RAmont-RAval distance

- Define the distance to sum.
- Append the distance_chenal_piezo dataset with the summed.

In [ ]:
# Rows to sum
RAmont_RAval = ['RAmont_RLMN', 'RLMN_RHIJK', 'RHIJK_RDEFG', 'RDEFG_RABC', 'RABC_RAval']
RAmont_RABC = ['RAmont_RLMN', 'RLMN_RHIJK', 'RHIJK_RDEFG', 'RDEFG_RABC']
RAmont_RDEFG = ['RAmont_RLMN', 'RLMN_RHIJK', 'RHIJK_RDEFG']
RAmont_RHIJK = ['RAmont_RLMN', 'RLMN_RHIJK']
# create lines to append
RAmont_distance = pd.DataFrame([['RAmont_RAval', sum(distance_chenal_piezo[distance_chenal_piezo.name.isin(RAmont_RAval)].distance_m)],
                                ['RAmont_RABC', sum(distance_chenal_piezo[distance_chenal_piezo.name.isin(RAmont_RABC)].distance_m)],
                                ['RAmont_RDEFG', sum(distance_chenal_piezo[distance_chenal_piezo.name.isin(RAmont_RDEFG)].distance_m)],
                                ['RAmont_RHIJK', sum(distance_chenal_piezo[distance_chenal_piezo.name.isin(RAmont_RHIJK)].distance_m)]], 
                                columns = ['name', 'distance_m'])
# concatenate the two dataframes
distance_chenal_piezo = pd.concat([distance_chenal_piezo, RAmont_distance])

## Rearrange piezometer level dataset to keep only level_ngf_m

- Combine for each piezo in piezo_dict to keep only level_ngf_m as level and date_time.
- The column 'name' will be used to identify the piezometer

In [ ]:
# combine for each piezo in piezo_dict to keep only level_ngf_m as level and date_time. The column 'name' will be used to identify the piezometer
df = pd.DataFrame()
for k, v in df_piezo_dict.items():
    v['name'] = k
    df = pd.concat([df, v], axis=0, ignore_index=True)
df = df[['date_time', 'level_ngf_m', 'name']]
df = df.sort_values(by='date_time', ascending=True)
df = df.rename(columns={'date_time':'dt', 'level_ngf_m': 'level'})
# add the coordinates of the piezometer with cols Est = x and Nord = y by merging with piezo_coords and using Point = name
df = pd.merge(df, piezo_coords, left_on='name', right_on='Point', how='left')
df = df.rename(columns={'Est': 'x', 'Nord': 'y', 'Alti': 'z'})
df = df[['dt', 'level', 'name', 'x', 'y', 'z']]

## Create a piezometer level dataset coordinate and compute distance

In [ ]:
# df z elevation piezometer
df_z = df.copy()
# group by name and get the first value of x, y and z
df_z = df_z.groupby('name').first().reset_index()
df_z = df_z[['name', 'x', 'y', 'z']]

In [ ]:
# create dateframe with distance between piezometer A-C, D-G, H-K, L-N
df_dist = pd.DataFrame({'name1': ['A', 'B', 'D', 'E', 'F', 'H', 'I', 'J', 'L', 'M'], 
                        'name2': ['B', 'C', 'E', 'F', 'G', 'I', 'J', 'K', 'M', 'N'], 'dist': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]})
df_dist = pd.merge(df_dist, df_z, left_on='name1', right_on='name', how='left')
df_dist = pd.merge(df_dist, df_z, left_on='name2', right_on='name', how='left')
df_dist['dist'] = np.sqrt((df_dist['x_x'] - df_dist['x_y'])**2 + (df_dist['y_x'] - df_dist['y_y'])**2)
df_dist = df_dist[['name1', 'name2', 'dist']]

## River level for each piezometer section and slope from nearest piezometer

- Pivot piezometer level dataset, remove rows without RAmont or RAval level.
- Calculate slope between on section.
- Compute bank river level for each piezometer section by distance fraction between RAmont and RAval.
- Get bank coordinate and distance from nearest section piezometer by date.
- Compute slope between nearest piezometer and bank river level

In [ ]:
# pivot table to have piezometer as columns
df_pivot = df.pivot_table(index='dt', columns='name', values='level')

# remove line if RAmont or RAval are NaN
df_pivot = df_pivot.dropna(subset=['RAmont', 'RAval'])

# calculate slope between piezometer
for couple in df_dist.iterrows():
    df_pivot[couple[1]['name2']+'-'+couple[1]['name1']] = (df_pivot[couple[1]['name2']] - df_pivot[couple[1]['name1']]) / couple[1]['dist']

## River level for each piezometer section and slope from nearest piezometer

- Compute bank river level for each piezometer section by distance fraction between RAmont and RAval.
- Get bank coordinate and distance from nearest section piezometer by the previous date (and not the closest date!).
- Compute slope between nearest piezometer and bank river level

In [ ]:
piezo_river = [['C', 'RABC'], ['G', 'RDEFG'], ['K', 'RHIJK'], ['N', 'RLMN']]

# Compute river water level for each transect
diff_RAmont_RAval = df_pivot['RAmont'] - df_pivot['RAval']
# get the distance between RAmont and RAval
dist_RAmont_RAval = distance_chenal_piezo.loc[distance_chenal_piezo['name'] == 'RAmont_RAval', 'distance_m'].values[0]

for piezo, river in piezo_river:
    # Precompute the distance for each transect to avoid recalculating
    dist_RAmont_transect = distance_chenal_piezo.loc[distance_chenal_piezo['name'] == f'RAmont_{river}', 'distance_m'].values[0]
    
    # Calculate the river column in one vectorized operation
    df_pivot[river] = df_pivot['RAmont'] - (diff_RAmont_RAval * (dist_RAmont_transect / dist_RAmont_RAval))

    # Prepare slope column and pre-filter berges_piezo_river and df_z_piezo for efficiency
    df_pivot[f'{river}-{piezo}'] = np.nan
    berges_piezo_river = berges_piezo[berges_piezo['name'] == river].sort_values(by="date")
    df_z_piezo = df_z[df_z['name'] == piezo]

    # Precompute piezo coordinates for distance calculation
    piezo_x, piezo_y = df_z_piezo['x'].iloc[0], df_z_piezo['y'].iloc[0]

    for i, target_date in enumerate(df_pivot.index):
        # Filter x_y_river only once per date
        x_y_river = berges_piezo_river[(berges_piezo_river['date'] <= target_date) & (berges_piezo_river['date'].shift(-1) > target_date)]
        
        if not x_y_river.empty:
            # Compute the distance only once for each valid x_y_river
            dist_piezo_river = np.sqrt((piezo_x - x_y_river['x'].iloc[0])**2 + (piezo_y - x_y_river['y'].iloc[0])**2)
            # Compute slope, use .at[] for single cell assignment to avoid multiple lookups
            df_pivot.at[target_date, f'{river}-{piezo}'] = (df_pivot.at[target_date, river] - df_pivot.at[target_date, piezo]) / dist_piezo_river


In [ ]:
df_pivot[['RAmont', 'RAval', 'RABC', 'RDEFG', 'RHIJK', 'RLMN']]

## Make plotly graph

In [ ]:
# Make plotly graph for RABC-C slope and RABC level on secondary y-axis with add_trace
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot['RABC-C'], mode='lines', name='RABC-C slope', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot['RABC'], mode='lines', name='RABC level', line=dict(color='red'), yaxis='y2'))
fig.update_layout(yaxis=dict(title='slope (m/m)', side='left'), yaxis2=dict(title='level (m)', side='right', overlaying='y', showgrid=False))
fig.show()


In [ ]:
# Make plotly graph for RDEFG-G slope and RDEFG level on secondary y-axis with add_trace
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot['RDEFG-G'], mode='lines', name='RDEFG-G slope', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot['RDEFG'], mode='lines', name='RDEFG level', line=dict(color='red'), yaxis='y2'))
fig.update_layout(yaxis=dict(title='slope (m/m)', side='left'), yaxis2=dict(title='level (m)', side='right', overlaying='y', showgrid=False))
fig.show()

In [ ]:
# Make plotly graph for RHIJK-K slope and RHIJK level on secondary y-axis with add_trace
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot['RHIJK-K'], mode='lines', name='RHIJK-K slope', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot['RHIJK'], mode='lines', name='RHIJK level', line=dict(color='red'), yaxis='y2'))
fig.update_layout(yaxis=dict(title='slope (m/m)', side='left'), yaxis2=dict(title='level (m)', side='right', overlaying='y', showgrid=False))
fig.show()

In [ ]:
# Make plotly graph for RLMN-N slope and RLMN level on secondary y-axis with add_trace
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot['RLMN-N'], mode='lines', name='RLMN-N slope', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df_pivot.index, y=df_pivot['RLMN'], mode='lines', name='RLMN level', line=dict(color='red'), yaxis='y2'))
fig.update_layout(yaxis=dict(title='slope (m/m)', side='left'), yaxis2=dict(title='level (m)', side='right', overlaying='y', showgrid=False))
fig.show()

In [ ]:
# 
df_pivot_2 = df_pivot.copy()
df_pivot_2 = df_pivot_2[30000:30003].loc[:, df_pivot_2.columns.str.contains('C|RAmont|RAval')]
df_pivot_2['RABC'] = np.nan
df_pivot_2['C-RABC'] = np.nan
for i in range(0, len(df_pivot_2)):
    print(i)
    print(df_pivot_2['RAmont'].iloc[i])
    diff_RAmont_RAval = df_pivot_2['RAmont'].iloc[i]-df_pivot_2['RAval'].iloc[i]
    dist_RAmont_RABC = distance_chenal_piezo[distance_chenal_piezo['name'].isin(['RAmont_RABC'])].distance_m.values
    dist_RAmont_RAval = distance_chenal_piezo[distance_chenal_piezo['name'].isin(['RAmont_RAval'])].distance_m.values
    df_pivot_2.loc[df_pivot_2.index[i], 'RABC'] = df_pivot_2['RAmont'].iloc[i]-(diff_RAmont_RAval)*(dist_RAmont_RABC/dist_RAmont_RAval)
    # slope
    berges_piezo_RABC = berges_piezo[berges_piezo.name.isin(['RABC'])].sort_values(by="date")
    df_z_c = df_z[df_z.name.isin(['C'])]
    target_date = df_pivot_2.index.values[i]
    x_y_rabc = berges_piezo_RABC[(berges_piezo_RABC['date'] <= target_date) & (berges_piezo_RABC['date'].shift(-1) > target_date)]
    dist_c_rabc = np.sqrt((df_z_c['x'].iloc[0] - x_y_rabc['x'].iloc[0])**2 + (df_z_c['y'].iloc[0] - x_y_rabc['y'].iloc[0])**2)
    df_pivot_2.loc[df_pivot_2.index[i], 'C-RABC'] = (df_pivot_2['C'].iloc[i] - df_pivot_2['RABC'].iloc[i])/dist_c_rabc

df_pivot_2

In [ ]:
berges_piezo[berges_piezo.name.isin(['RABC'])].sort_values(by="date")

In [ ]:
berges_piezo_ABC = berges_piezo[berges_piezo.name.isin(['RABC'])].sort_values(by="date")
df_z_C = df_z[df_z.name.isin(['C'])]
target_date = df_pivot_2.index.values[1]
# berges_piezo_ABC['date']<= target_date

# target_date = pd.to_datetime("2024-04-25")


interval = berges_piezo_ABC[(berges_piezo_ABC['date'] <= target_date) & (berges_piezo_ABC['date'].shift(-1) > target_date)]

# df_pivot_2['C'].iloc[1] 
# interval['x'].iloc[0]
dist_C_RABC = np.sqrt((df_z_C['x'].iloc[0] - interval['x'].iloc[0])**2 + (df_z_C['y'].iloc[0] - interval['y'].iloc[0])**2)
slope_c_rabc = df_pivot_2['C'].iloc[1] - 
# interval

In [ ]:
df_z[df_z.name.isin(['C'])]['x']

In [ ]:
# calculate slope between A and C for each day
df_slope = df.copy()
df_slope = df_slope[df_slope['name'].isin(['A', 'C'])]
df_slope = df_slope.pivot(index='dt', columns='name', values='level')
# calculate slope between A and C
df_slope['slope'] = (df_slope['C'] - df_slope['A']) / df_dist.loc[0, 'dist']*100
df_slope = df_slope.reset_index()

# plot C and A level on the first y axis and the slope on the second y axis, all on the same graph with two y axis
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()
ax1.plot(df_slope['dt'], df_slope['A'], 'g-')
ax1.plot(df_slope['dt'], df_slope['C'], 'r-')
ax2.plot(df_slope['dt'], df_slope['slope'], 'b-')
# add labels
ax1.set_xlabel('Date')
ax1.set_ylabel('A and C level [m]', color='black')
ax2.set_ylabel('Slope A-C [%]', color='black')
# add legend without overlapping
ax1.legend(['A', 'C'], loc='upper left')
ax2.legend(['Slope A-C'], loc='upper right')
# add an horizontal line at 0 slope
ax2.axhline(y=0, color='black', linestyle='--')
plt.show()


In [ ]:
# create dataframe with only A piezometer
df_A = df[df['name'] == 'A']
# remove value if level variation is greater than 1m
df_A['level_diff'] = df_A['level'].diff()
# if level_diff is greater than 1, replace level by NaN
# df_A[df_A['level_diff'] < -1]
df_A.loc[df_A['level_diff'] < -1, 'level'] = np.nan
# if level_diff is greater than 1, replace level_diff by NaN
df_A.loc[df_A['level_diff'] < -1, 'level_diff'] = np.nan
df_A.loc[df_A['level_diff'] > 1, 'level_diff'] = np.nan
# for NaN values in level, interpolate linearly with the previous and next non NaN values
df_A['level'] = df_A['level'].interpolate(method='linear')
df_A['level_diff'] = df_A['level_diff'].interpolate(method='linear')
df_A = df_A[df_A['dt'] > '2023-01-31 10:00:00']
df_A







In [ ]:
df_A = df[df['name'] == 'A']
df_A

In [ ]:
def process_piezometer_data(df, piezometer_name, time_filter):
    # Filter dataframe for the specified piezometer
    df_filtered = df[df['name'] == piezometer_name].copy()
    
    # Calculate the difference in level
    df_filtered['level_diff'] = df_filtered['level'].diff()
    
    # Replace level and level_diff with NaN if level_diff is outside the specified range
    df_filtered.loc[df_filtered['level_diff'] < -1, 'level'] = np.nan
    df_filtered.loc[df_filtered['level_diff'] < -1, 'level_diff'] = np.nan
    df_filtered.loc[df_filtered['level_diff'] > 1, 'level_diff'] = np.nan
    
    # Interpolate NaN values in 'level' and 'level_diff' columns
    df_filtered['level'] = df_filtered['level'].interpolate(method='linear')
    df_filtered['level_diff'] = df_filtered['level_diff'].interpolate(method='linear')
    
    # Filter by the given timestamp
    df_filtered = df_filtered[df_filtered['dt'] > time_filter]
    
    return df_filtered

# Process data for piezometer A
df_C = process_piezometer_data(df, 'C', '2023-01-31 10:00:00')

In [ ]:
df_C


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Scatter(x=df_A['dt'], y=df_A['level'], name="level"),
    secondary_y=False,
)

fig.add_trace(
    go.Scatter(x=df_A['dt'], y=df_A['level_diff'], name="level diff"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="A piezometer level and level variation"
)

# Set x-axis title
fig.update_xaxes(title_text="xaxis title")

# Set y-axes titles
fig.update_yaxes(title_text="level (m)", secondary_y=False)
fig.update_yaxes(
    title_text="Level variation (m)", 
    secondary_y=True, 
    range=[-0.15, 0.15],        # Set the range of the secondary axis
    side='right',          # Keep both axes on the left
    showgrid=False,       # Hide the grid for the secondary axis
)

fig.show()

In [ ]:
# plot A level piezometer
fig, ax = plt.subplots()
df_A = df[df['name'] == 'A']
ax.plot(df_A['dt'], df_A['level'])
ax.set_xlabel('Date')
ax.set_ylabel('A level [m]')
plt.show()

In [ ]:
# plot all the piezometer level
fig, ax = plt.subplots()
for name in df['name'].unique():
    df_p = df[df['name'] == name]
    ax.plot(df_p['dt'], df_p['level'], label=name)
ax.set_xlabel('Date')
ax.set_ylabel('Level [m]')
ax.legend(loc='upper left')
plt.show()
